In [1]:
import itertools
import gurobipy as gp
import numpy as np
import pandas as pd


p = 0.625
pi = {}
r = 1.5


# Define the possible outcomes for lambda(ikts)
outcomes = [10, 20]

# Generate all possible combinations of outcomes for each stage
scenarios = {}
scenario_num = 1
for scenario in itertools.product(outcomes, repeat=4):
    scenarios[f"Scenario {scenario_num}"] = list(scenario)
    scenario_num += 1

# # Print the generated lambda_scenarios
# for scenario, values in lambda_scenarios.items():
#     print(f"{scenario}: {values}")
scenarios

{'Scenario 1': [10, 10, 10, 10],
 'Scenario 2': [10, 10, 10, 20],
 'Scenario 3': [10, 10, 20, 10],
 'Scenario 4': [10, 10, 20, 20],
 'Scenario 5': [10, 20, 10, 10],
 'Scenario 6': [10, 20, 10, 20],
 'Scenario 7': [10, 20, 20, 10],
 'Scenario 8': [10, 20, 20, 20],
 'Scenario 9': [20, 10, 10, 10],
 'Scenario 10': [20, 10, 10, 20],
 'Scenario 11': [20, 10, 20, 10],
 'Scenario 12': [20, 10, 20, 20],
 'Scenario 13': [20, 20, 10, 10],
 'Scenario 14': [20, 20, 10, 20],
 'Scenario 15': [20, 20, 20, 10],
 'Scenario 16': [20, 20, 20, 20]}

In [2]:
# Given parameters and their corresponding costs
fixed_cap_cost = 240         # Corresponds to alpha
admission_cost = 5        # Corresponds to beta
transfer_cost = 10           # Corresponds to gamma
waiting_cost = {1: 50, 2: 20}  # Corresponds to delta_k (using dictionary for index k)
unmet_demand_cost = {1: 1000, 2: 500}  # Corresponds to eta_k (using dictionary for index k)
l = 7
B = 10000


In [3]:
# Create a list to store solutions
solutions_0 = []

# Loop over scenarios
for scenario, d_values in scenarios.items():
    # Create a new optimization model
    model = gp.Model()
    
    # Define ranges and parameters
    # Parameters
    I = range(1, 4)  # Set I is {1, 2, 3, 4}
    K = [1, 2]       # Set K is {1, 2}
    T = range(1, 5)  # Set T is {1, 2, 3, 4}
    S = range(1, 17)

    # Define decision variables with appropriate indices and integer type
    x = model.addVars(I, name="x", vtype=gp.GRB.INTEGER)
    w = model.addVars(I, K, T, name="w", vtype=gp.GRB.INTEGER)
    y = model.addVars(I, K, T, name="y", vtype=gp.GRB.INTEGER)
    f = model.addVars(I, I, T, name="f", vtype=gp.GRB.INTEGER) 
    u = model.addVars(I, K, T, name="u", vtype=gp.GRB.INTEGER)
    z = model.addVars(I, K, T, name="z", vtype=gp.GRB.INTEGER)
    c = model.addVars(I, T, name="c", vtype=gp.GRB.INTEGER)

    
    # Update the model to include new variables
    model.update()


    # Objective function
    obj = gp.quicksum(fixed_cap_cost * x[i] for i in I) + \
        gp.quicksum(admission_cost * y[i, k, t] for i in I for k in K for t in T) + \
        gp.quicksum(transfer_cost * f[i, j, t] for i in I for j in I for t in T if i != j) + \
        gp.quicksum(waiting_cost[k] * w[i, k, t] for i in I for k in K for t in T) + \
        gp.quicksum(unmet_demand_cost[k] * u[i, k, t] for i in I for k in K for t in T)

    model.setObjective(obj, gp.GRB.MINIMIZE)


    # Add constraints

    for i in I:
        for t in range(2, len(T)+1):
            model.addConstr((w[i, 1, t] + y[i, 1, t] + u[i, 1, t] == w[i, 1, t-1] + 0.3 * d_values[t-1]), name="Const1")

    for i in I:
        for t in range(2, len(T)+1):
            model.addConstr((gp.quicksum(f[i, j, t] for j in I if j!= i) + y[i, 2, t] + u[i, 2, t] == w[i, 2, t-1] + 0.7 * d_values[t-1] + gp.quicksum(f[j, i, t-1] for j in I if j != i)), name="Const2")
    
    for i in I:
        model.addConstr((w[i, 1, 1] + y[i, 1, 1] + u[i, 1, 1] == 0.3 * d_values[1]) , name="Const3")
        model.addConstr((gp.quicksum(f[i, j, t] for j in I if j!= i) + w[i, 2, 1] + y[i, 2, 1] + u[i, 2, 1] == 0.7 * d_values[1]), name="Const4")
    
    for i in I:
        for t in T:
            model.addConstr((gp.quicksum(y[i, k, t] for k in K) <= c[i, t]), name="Const5")

    for i in I:
        for t in range(2, len(T)+1):
            model.addConstr((c[i, t] <= x[i]), name="Const6")
    for i in I:
        model.addConstr((c[i, 1] == x[i]), name="Const7")

    for i in I:
        for t in range(2, len(T)+1):
            model.addConstr((c[i, t] == c[i, t-1] - gp.quicksum(y[i, k, t-1] for k in K) + gp.quicksum(z[i, k, t-1] for k in K)), name="Const8")

    for i in I:
        for t in range(2, len(T)+1):
            for k in K:
                model.addConstr((z[i, k, t] <= y[i, k, t-1]), name="Const9")

    for i in I:
        for k in K:
            model.addConstr((z[i, k, 1] == 0), name="Const10")

    model.addConstr((gp.quicksum(fixed_cap_cost * x[i] for i in I) <= B), name="Conts11")

    for i in I:
        for j in I:
            for t in T:
                model.addConstr((f[i, j, t] <= l), name="Const12")


    
                # model.addConstr((f[i, j, t-1] <= c[j, t]), name="Const13")
    
    model.optimize()


    solution_0 = {
        "Scenario": scenario,
        "Status": model.status,
        "Objective": model.getObjective().getValue(),
        "Values": {
            "x": {i: x[i].x for i in I},
            "w": {(i, k, t): w[i, k, t].x for i in I for k in K for t in T},
            "y": {(i, k, t): y[i, k, t].x for i in I for k in K for t in T},
            "f": {(i, j, t): f[i, j, t].x for i in I for j in I for t in T},
            "u": {(i, k, t): u[i, k, t].x for i in I for k in K for t in T},
            "z": {(i, k, t): z[i, k, t].x for i in I for k in K for t in T},
            "c": {(i, t): c[i, t].x for i in I for t in T}
        }
    }
    solutions_0.append(solution_0)


Restricted license - for non-production use only - expires 2025-11-24
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i5-8250U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 118 rows, 147 columns and 318 nonzeros
Model fingerprint: 0x63ac8e97
Variable types: 0 continuous, 147 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [5e+00, 1e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+00, 1e+04]
Found heuristic solution: objective 69450.000000
Presolve removed 57 rows and 66 columns
Presolve time: 0.00s
Presolved: 61 rows, 81 columns, 204 nonzeros
Found heuristic solution: objective 26115.000000
Variable types: 0 continuous, 81 integer (0 binary)

Root relaxation: objective 9.360000e+03, 58 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    | 

In [4]:
solutions_0

[{'Scenario': 'Scenario 1',
  'Status': 2,
  'Objective': 9360.0,
  'Values': {'x': {1: 11.0, 2: 13.0, 3: 6.0},
   'w': {(1, 1, 1): 0.0,
    (1, 1, 2): 0.0,
    (1, 1, 3): 3.0,
    (1, 1, 4): 6.0,
    (1, 2, 1): 0.0,
    (1, 2, 2): 0.0,
    (1, 2, 3): 0.0,
    (1, 2, 4): -0.0,
    (2, 1, 1): 0.0,
    (2, 1, 2): 0.0,
    (2, 1, 3): 3.0,
    (2, 1, 4): 6.0,
    (2, 2, 1): 0.0,
    (2, 2, 2): 0.0,
    (2, 2, 3): 0.0,
    (2, 2, 4): -0.0,
    (3, 1, 1): 0.0,
    (3, 1, 2): 0.0,
    (3, 1, 3): 3.0,
    (3, 1, 4): 6.0,
    (3, 2, 1): 0.0,
    (3, 2, 2): 0.0,
    (3, 2, 3): 0.0,
    (3, 2, 4): -0.0},
   'y': {(1, 1, 1): 3.0,
    (1, 1, 2): 3.0,
    (1, 1, 3): 0.0,
    (1, 1, 4): 0.0,
    (1, 2, 1): 0.0,
    (1, 2, 2): 5.0,
    (1, 2, 3): 3.0,
    (1, 2, 4): 8.0,
    (2, 1, 1): 3.0,
    (2, 1, 2): 3.0,
    (2, 1, 3): 0.0,
    (2, 1, 4): 0.0,
    (2, 2, 1): 0.0,
    (2, 2, 2): 7.0,
    (2, 2, 3): 3.0,
    (2, 2, 4): 10.0,
    (3, 1, 1): 3.0,
    (3, 1, 2): 3.0,
    (3, 1, 3): 0.0,
    (3, 1, 4)

In [5]:
# Printing the first setting of pi(lagrangian multiplier) which is set to 0 initially
for i in I:
    for t in T:
        for s in S:
            pi[(i,t,s)] = 0

print(pi)

{(1, 1, 1): 0, (1, 1, 2): 0, (1, 1, 3): 0, (1, 1, 4): 0, (1, 1, 5): 0, (1, 1, 6): 0, (1, 1, 7): 0, (1, 1, 8): 0, (1, 1, 9): 0, (1, 1, 10): 0, (1, 1, 11): 0, (1, 1, 12): 0, (1, 1, 13): 0, (1, 1, 14): 0, (1, 1, 15): 0, (1, 1, 16): 0, (1, 2, 1): 0, (1, 2, 2): 0, (1, 2, 3): 0, (1, 2, 4): 0, (1, 2, 5): 0, (1, 2, 6): 0, (1, 2, 7): 0, (1, 2, 8): 0, (1, 2, 9): 0, (1, 2, 10): 0, (1, 2, 11): 0, (1, 2, 12): 0, (1, 2, 13): 0, (1, 2, 14): 0, (1, 2, 15): 0, (1, 2, 16): 0, (1, 3, 1): 0, (1, 3, 2): 0, (1, 3, 3): 0, (1, 3, 4): 0, (1, 3, 5): 0, (1, 3, 6): 0, (1, 3, 7): 0, (1, 3, 8): 0, (1, 3, 9): 0, (1, 3, 10): 0, (1, 3, 11): 0, (1, 3, 12): 0, (1, 3, 13): 0, (1, 3, 14): 0, (1, 3, 15): 0, (1, 3, 16): 0, (1, 4, 1): 0, (1, 4, 2): 0, (1, 4, 3): 0, (1, 4, 4): 0, (1, 4, 5): 0, (1, 4, 6): 0, (1, 4, 7): 0, (1, 4, 8): 0, (1, 4, 9): 0, (1, 4, 10): 0, (1, 4, 11): 0, (1, 4, 12): 0, (1, 4, 13): 0, (1, 4, 14): 0, (1, 4, 15): 0, (1, 4, 16): 0, (2, 1, 1): 0, (2, 1, 2): 0, (2, 1, 3): 0, (2, 1, 4): 0, (2, 1, 5): 0, (2, 1

In [6]:
Cbar = {}

# Initialize Cbar for each scenario with zero
for scenario in scenarios.keys():
    Cbar[scenario] = 0

# Assume p is the probability for each scenario, already defined
for solution in solutions_0:
    scenario = solution["Scenario"]
    solution_values = solution["Values"]
    # Accumulate the weighted sum of c for each i,t
    for i in I:
        for t in T:
            c_hat_its = solution_values['c'][(i, t)]
            Cbar[scenario] += p * c_hat_its

# # Now print the accumulated Cbar values for each scenario
# for scenario, cbar_val in Cbar.items():
#     print(f"{scenario}: {cbar_val}")


# Create a new dictionary with integer keys
Cbar_new= {int(scenario.split()[1]): value for scenario, value in Cbar.items()}

In [7]:
Cbar_new

{1: 50.625,
 2: 71.25,
 3: 71.25,
 4: 71.25,
 5: 61.875,
 6: 65.625,
 7: 65.625,
 8: 76.875,
 9: 50.625,
 10: 71.25,
 11: 71.25,
 12: 71.25,
 13: 61.875,
 14: 65.625,
 15: 65.625,
 16: 76.875}

In [8]:
max_it = 20
all_solutions = {}
all_Cbars = {}
K = 5
for k in range(1, max_it):
    print("****************************Iteration:", k)
    solutions = []
    # Loop over scenarios
    for scenario, d_values in scenarios.items():
        # Create a new optimization model
        model = gp.Model()
        
        # Define ranges and parameters
        # Parameters
        I = range(1, 4)  # Set I is {1, 2, 3, 4}
        K = [1, 2]       # Set K is {1, 2}
        T = range(1, 5)  # Set T is {1, 2, 3, 4}
        S = range(1, 17)

        # Define decision variables with appropriate indices and integer type
        x = model.addVars(I, name="x", vtype=gp.GRB.INTEGER)
        w = model.addVars(I, K, T, name="w", vtype=gp.GRB.INTEGER)
        y = model.addVars(I, K, T, name="y", vtype=gp.GRB.INTEGER)
        f = model.addVars(I, I, T, name="f", vtype=gp.GRB.INTEGER) 
        u = model.addVars(I, K, T, name="u", vtype=gp.GRB.INTEGER)
        z = model.addVars(I, K, T, name="z", vtype=gp.GRB.INTEGER)
        c = model.addVars(I, T, name="c", vtype=gp.GRB.INTEGER)

        
        # Update the model to include new variables
        model.update()


        # Objective function
        obj = gp.quicksum(fixed_cap_cost * x[i] for i in I) + \
            gp.quicksum(admission_cost * y[i, k, t] for i in I for k in K for t in T) + \
            gp.quicksum(transfer_cost * f[i, j, t] for i in I for j in I for t in T if i != j) + \
            gp.quicksum(waiting_cost[k] * w[i, k, t] for i in I for k in K for t in T) + \
            gp.quicksum(unmet_demand_cost[k] * u[i, k, t] for i in I for k in K for t in T) + \
            gp.quicksum(pi[(i, t, s)] * c[i, t] for i in I for t in T for s in S) + \
            gp.quicksum(r/2 * ((c[i, t] - Cbar_new[i]) * (c[i, t] - Cbar_new[i])) for i in I for t in T)

        model.setObjective(obj, gp.GRB.MINIMIZE)

        for i in I:
            for t in range(2, len(T)+1):
                model.addConstr((w[i, 1, t] + y[i, 1, t] + u[i, 1, t] == w[i, 1, t-1] + 0.3 * d_values[t-1]), name="Const1")

        for i in I:
            for t in range(2, len(T)+1):
                model.addConstr((gp.quicksum(f[i, j, t] for j in I if j!= i) + y[i, 2, t] + u[i, 2, t] == w[i, 2, t-1] + 0.7 * d_values[t-1] + gp.quicksum(f[j, i, t-1] for j in I if j != i)), name="Const2")
        
        for i in I:
            model.addConstr((w[i, 1, 1] + y[i, 1, 1] + u[i, 1, 1] == 0.3 * d_values[1]) , name="Const3")
            model.addConstr((gp.quicksum(f[i, j, t] for j in I if j!= i) + w[i, 2, 1] + y[i, 2, 1] + u[i, 2, 1] == 0.7 * d_values[1]), name="Const4")
        
        for i in I:
            for t in T:
                model.addConstr((gp.quicksum(y[i, k, t] for k in K) <= c[i, t]), name="Const5")

        for i in I:
            for t in range(2, len(T)+1):
                model.addConstr((c[i, t] <= x[i]), name="Const6")
        for i in I:
            model.addConstr((c[i, 1] == x[i]), name="Const7")

        for i in I:
            for t in range(2, len(T)+1):
                model.addConstr((c[i, t] == c[i, t-1] - gp.quicksum(y[i, k, t-1] for k in K) + gp.quicksum(z[i, k, t-1] for k in K)), name="Const8")

        for i in I:
            for t in range(2, len(T)+1):
                for k in K:
                    model.addConstr((z[i, k, t] <= y[i, k, t-1]), name="Const9")

        for i in I:
            for k in K:
                model.addConstr((z[i, k, 1] == 0), name="Const10")

        model.addConstr((gp.quicksum(fixed_cap_cost * x[i] for i in I) <= B), name="Conts11")

        for i in I:
            for j in I:
                for t in T:
                    model.addConstr((f[i, j, t] <= l), name="Const12")
        
        model.optimize()

        # Store solution
        solution = {
            "Scenario": scenario,
            "Status": model.status,
            "Objective": model.getObjective().getValue(),
            "Values": {
            "x": {i: x[i].x for i in I},
            "w": {(i, k, t): w[i, k, t].x for i in I for k in K for t in T},
            "y": {(i, k, t): y[i, k, t].x for i in I for k in K for t in T},
            "f": {(i, j, t): f[i, j, t].x for i in I for j in I for t in T},
            "u": {(i, k, t): u[i, k, t].x for i in I for k in K for t in T},
            "z": {(i, k, t): z[i, k, t].x for i in I for k in K for t in T},
            "c": {(i, t): c[i, t].x for i in I for t in T}}
        }
        solutions.append(solution)

    all_solutions[k] = solutions

    Cbar= {}
    for solution in solutions:
        scenario = solution["Scenario"]
        solution_values = solution["Values"]
        # Accumulate the weighted sum of c for each i,t
        Cbar[scenario] = 0
        for i in I:
            for t in T:
                c_hat_its = solution_values['c'][(i, t)]
                Cbar[scenario] += p * c_hat_its
    # # Create a new dictionary with integer keys
    Cbar_new= {int(scenario.split()[1]): value for scenario, value in Cbar.items()} 

    all_Cbars[k] = Cbar_new


****************************Iteration: 1
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i5-8250U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 118 rows, 147 columns and 318 nonzeros
Model fingerprint: 0x5941da97
Model has 12 quadratic objective terms
Variable types: 0 continuous, 147 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [5e+00, 1e+03]
  QObjective range [2e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+00, 1e+04]
Found heuristic solution: objective 1.200001e+11
Presolve removed 57 rows and 66 columns
Presolve time: 0.00s
Presolved: 61 rows, 81 columns, 204 nonzeros
Presolved model has 12 quadratic objective terms
Found heuristic solution: objective 74334.234375
Variable types: 0 continuous, 81 integer (0 binary)
Found heuristic solution: object

In [14]:
iteration

2

In [9]:
all_solutions

{2: [{'Scenario': 'Scenario 1',
   'Status': 2,
   'Objective': 56874.703125,
   'Values': {'x': {1: 16.0, 2: 13.0, 3: 12.0},
    'w': {(1, 1, 1): -0.0,
     (1, 1, 2): -0.0,
     (1, 1, 3): 3.0,
     (1, 1, 4): 4.0,
     (1, 2, 1): -0.0,
     (1, 2, 2): 0.0,
     (1, 2, 3): 0.0,
     (1, 2, 4): -0.0,
     (2, 1, 1): -0.0,
     (2, 1, 2): -0.0,
     (2, 1, 3): 3.0,
     (2, 1, 4): 6.0,
     (2, 2, 1): -0.0,
     (2, 2, 2): 0.0,
     (2, 2, 3): 0.0,
     (2, 2, 4): -0.0,
     (3, 1, 1): -0.0,
     (3, 1, 2): -0.0,
     (3, 1, 3): 3.0,
     (3, 1, 4): 6.0,
     (3, 2, 1): -0.0,
     (3, 2, 2): 0.0,
     (3, 2, 3): 0.0,
     (3, 2, 4): -0.0},
    'y': {(1, 1, 1): 3.0,
     (1, 1, 2): 3.0,
     (1, 1, 3): -0.0,
     (1, 1, 4): 2.0,
     (1, 2, 1): 0.0,
     (1, 2, 2): -0.0,
     (1, 2, 3): -0.0,
     (1, 2, 4): 14.0,
     (2, 1, 1): 3.0,
     (2, 1, 2): 3.0,
     (2, 1, 3): -0.0,
     (2, 1, 4): -0.0,
     (2, 2, 1): 0.0,
     (2, 2, 2): 1.0,
     (2, 2, 3): -0.0,
     (2, 2, 4): 13.0,
   

In [10]:
# Extract objective functions from all_solutions dictionary
objective_functions = []

for iteration, solutions in all_solutions.items():
    objective_values = []
    for solution in solutions:
        objective_values.append(solution['Objective'])
    objective_functions.append(objective_values)

print(objective_functions)


[[56874.703125, 59719.765625, 60459.453125, 71119.765625, 60936.203125, 62659.078125, 65539.390625, 68316.078125, 56874.703125, 59719.765625, 60459.453125, 71119.765625, 60936.203125, 62659.078125, 65539.390625, 68316.078125]]


In [11]:
import pandas as pd

# Initialize lists to store iterations and objective values
iterations = []
objective_values = []

# Iterate over all_solutions dictionary
for iteration, solutions in all_solutions.items():
    for solution in solutions:
        # Append iteration number and objective value to respective lists
        objective_values.append(solution['Objective'])

# Create a DataFrame from the lists
df = pd.DataFrame({
    'Objective_Values': objective_values
})

print(df)


    Objective_Values
0       56874.703125
1       59719.765625
2       60459.453125
3       71119.765625
4       60936.203125
5       62659.078125
6       65539.390625
7       68316.078125
8       56874.703125
9       59719.765625
10      60459.453125
11      71119.765625
12      60936.203125
13      62659.078125
14      65539.390625
15      68316.078125


In [12]:
# Display solutions
for iteration, solutions in all_solutions.items():
    print("Iteration:", iteration)
    for solution in solutions:
        print("Scenario:", solution["Scenario"])
        if solution["Status"] == 2:
            print("Objective:", solution["Objective"])
            print("Variable Values:")
            x_values = solution["Values"]["x"]
            for i, x_val in x_values.items():
                print("x_{} = {}".format(i, x_val))
            print("Other Variable Values:")
            for key, value in solution["Values"].items():
                if key != "x":
                    print(key, "=", value)
        else:
            print("No solution found")
        print()


Iteration: 2
Scenario: Scenario 1
Objective: 56874.703125
Variable Values:
x_1 = 16.0
x_2 = 13.0
x_3 = 12.0
Other Variable Values:
w = {(1, 1, 1): -0.0, (1, 1, 2): -0.0, (1, 1, 3): 3.0, (1, 1, 4): 4.0, (1, 2, 1): -0.0, (1, 2, 2): 0.0, (1, 2, 3): 0.0, (1, 2, 4): -0.0, (2, 1, 1): -0.0, (2, 1, 2): -0.0, (2, 1, 3): 3.0, (2, 1, 4): 6.0, (2, 2, 1): -0.0, (2, 2, 2): 0.0, (2, 2, 3): 0.0, (2, 2, 4): -0.0, (3, 1, 1): -0.0, (3, 1, 2): -0.0, (3, 1, 3): 3.0, (3, 1, 4): 6.0, (3, 2, 1): -0.0, (3, 2, 2): 0.0, (3, 2, 3): 0.0, (3, 2, 4): -0.0}
y = {(1, 1, 1): 3.0, (1, 1, 2): 3.0, (1, 1, 3): -0.0, (1, 1, 4): 2.0, (1, 2, 1): 0.0, (1, 2, 2): -0.0, (1, 2, 3): -0.0, (1, 2, 4): 14.0, (2, 1, 1): 3.0, (2, 1, 2): 3.0, (2, 1, 3): -0.0, (2, 1, 4): -0.0, (2, 2, 1): 0.0, (2, 2, 2): 1.0, (2, 2, 3): -0.0, (2, 2, 4): 13.0, (3, 1, 1): 3.0, (3, 1, 2): 3.0, (3, 1, 3): -0.0, (3, 1, 4): -0.0, (3, 2, 1): 0.0, (3, 2, 2): 2.0, (3, 2, 3): -0.0, (3, 2, 4): 12.0}
f = {(1, 1, 1): -0.0, (1, 1, 2): -0.0, (1, 1, 3): -0.0, (1, 1, 4): 

In [13]:
# Display solutions in a readable format
for iteration, solutions in all_solutions.items():
    print("Iteration:", iteration)
    
    for solution in solutions:
        print("Scenario:", solution["Scenario"])
        
        if solution["Status"] == 2:  # Check if a solution was found
            print("  Objective Value: {:.2f}".format(solution["Objective"]))
            
            # Display x variable values
            print("  Variable Values (x):")
            for i, x_val in sorted(solution["Values"]["x"].items()):
                print("    x_{} = {:.2f}".format(i, x_val))
            
            # Display other variable values
            print("  Other Variable Values:")
            for key, values in solution["Values"].items():
                if key != "x":
                    # Check if the value is a dictionary
                    if isinstance(values, dict):
                        for sub_key, sub_val in values.items():
                            print("    {}_{} = {:.2f}".format(key, sub_key, sub_val))
                    else:
                        print("    {} = {:.2f}".format(key, values))
        else:
            print("  No solution found.")
        
        print("-" * 40)  # Separator for readability


Iteration: 2
Scenario: Scenario 1
  Objective Value: 56874.70
  Variable Values (x):
    x_1 = 16.00
    x_2 = 13.00
    x_3 = 12.00
  Other Variable Values:
    w_(1, 1, 1) = -0.00
    w_(1, 1, 2) = -0.00
    w_(1, 1, 3) = 3.00
    w_(1, 1, 4) = 4.00
    w_(1, 2, 1) = -0.00
    w_(1, 2, 2) = 0.00
    w_(1, 2, 3) = 0.00
    w_(1, 2, 4) = -0.00
    w_(2, 1, 1) = -0.00
    w_(2, 1, 2) = -0.00
    w_(2, 1, 3) = 3.00
    w_(2, 1, 4) = 6.00
    w_(2, 2, 1) = -0.00
    w_(2, 2, 2) = 0.00
    w_(2, 2, 3) = 0.00
    w_(2, 2, 4) = -0.00
    w_(3, 1, 1) = -0.00
    w_(3, 1, 2) = -0.00
    w_(3, 1, 3) = 3.00
    w_(3, 1, 4) = 6.00
    w_(3, 2, 1) = -0.00
    w_(3, 2, 2) = 0.00
    w_(3, 2, 3) = 0.00
    w_(3, 2, 4) = -0.00
    y_(1, 1, 1) = 3.00
    y_(1, 1, 2) = 3.00
    y_(1, 1, 3) = -0.00
    y_(1, 1, 4) = 2.00
    y_(1, 2, 1) = 0.00
    y_(1, 2, 2) = -0.00
    y_(1, 2, 3) = -0.00
    y_(1, 2, 4) = 14.00
    y_(2, 1, 1) = 3.00
    y_(2, 1, 2) = 3.00
    y_(2, 1, 3) = -0.00
    y_(2, 1, 4) = -0